In [ ]:
import json
import numpy as np
import cv2

# ファイルパス
json_file_path = "./test_distortiuon_keypoints.json"

# ファイルを開き、JSONとして読み込む
with open(json_file_path, 'r') as f:
    image_points_dict = json.load(f)

# 画像ファイルのパス
image_file_path = "./test_distortion.png"

# 画像を読み込む
img = cv2.imread(image_file_path)

# 画像の幅と高さを取得
image_height, image_width = img.shape[:2]


# サッカーコートの寸法をメートル単位で指定 (例: 105m x 68m)
# ここで、ピッチの各点の実世界座標をメートル単位で計算します
world_points_dict = {}
for key, value in image_points_dict.items():
    # キーから実世界の座標を抽出 (例: "(0,0)" -> (0.0, 0.0))
    coord = tuple(map(float, key.strip("()").split(",")))
    world_points_dict[key] = [coord[0], coord[1], 0.0]  # Z座標は0

# numpy配列に変換
image_points = np.array(list(image_points_dict.values()), dtype=np.float32)
world_points = np.array(list(world_points_dict.values()), dtype=np.float32)

# image_points should be of shape (n, 1, 2)
image_points = image_points.reshape(-1, 1, 2)

# world_points should be of shape (n, 1, 3)
world_points = world_points.reshape(-1, 1, 3)

# カメラ行列と歪み係数の初期値
K = np.zeros((3, 3))
D = np.zeros((4, 1))

# フラグを設定 (魚眼カメラのモデルを使用)
flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_FIX_SKEW

# キャリブレーション
retval, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    [world_points], [image_points], (image_width, image_height), K, D, flags=flags
)

# 新しいカメラ行列を取得（alphaを調整して歪み補正後の画像の見切れを調整）
new_K, roi = cv2.getOptimalNewCameraMatrix(K, D, (image_width, image_height), alpha=1.02, centerPrincipalPoint=1)

# 歪み補正マップを初期化
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), new_K, (image_width, image_height), cv2.CV_16SC2)

# 歪み補正を適用
undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# 補正された画像をファイルに保存
cv2.imwrite('./undistorted_image.png', undistorted_img)

import matplotlib.pyplot as plt
# 表示する画像のサイズをインチ単位で設定（例: 幅15インチ、高さ5インチ）
plt.figure(figsize=(25, 10))

# 補正された画像を表示
plt.imshow(cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB))
plt.show()
